In [9]:
from spatialoperations import SpatialOperations
from spatialoperations.config import SpatialOpsConfig


config = SpatialOpsConfig(
    root="s3://cl-lake-test/sids-test-tough-topobathy",
    dx=0.00026977476123492776,
    epsg=4326,
    bounds=[-179.99, -50, 179.99, 50],
    chunk_size=5000,
)
so = SpatialOperations(config=config)
so

In [13]:
import geopandas as gpd
import numpy as np

countries = [
    "FSM_10",
    # "FSM_05",
    # "FSM_08",
    # "FSM_21",
    # "FSM_20",
    # "FSM_18",
    # "FSM_11",
    # "KIR_09"
]
gdf = gpd.read_file("/cccr-lab/001_projects/002_nbs-adapt/003_regions_study_units/All_Regions_Marina/02_Polygons_SIDS_20250701/Polygons_SIDS_20250701_final.gpkg")   
regions = gdf[
    gdf["Subreg_Name"].apply(lambda x: np.any([c in x for c in countries]))
]
regions











,region_id,HAS_REEFS,NumberOfCe,PolygonAre,ACADataAva,ACABenthic,ACABathyme,utmzone,utmcode,Subreg_Name,geometry
52,53,NaN,4.193852e+06,3.774467e+09,YES,YES,YES,56PN,32656,FSM_10,"MULTIPOLYGON (((150.21069 9.05281, 150.53484 8..."


In [14]:
from glob import glob
from spatialoperations.types.xrtypes import RasterLike
import os
import pandas as pd
from shapely.geometry import box

idxs=so.ro.zarr_manager.chunks_initialized(
    "ACA",
    "topobathy/bathymetry"
)
fabdem = glob('/cccr-lab/000_data-repository/008_fabdem/tiles/*.tif')

def path_to_bounds(path):
    fname = os.path.basename(path)
    ll = fname.split("_")[0]
    north = ll.startswith("N")
    east = "E" in ll
    
    split_on_y = "N" if north else "S"
    split_on_x = "E" if east else "W"
    
    y, x = ll.replace(split_on_y, "").split(split_on_x)
    x = float(x)
    y = float(y)
    
    if not north:
        y = -y
        
    if not east:
        x = -x
        
    return box(x, y, x+1, y+1)

df = pd.DataFrame(fabdem, columns=["path"])
df["bounds"] = df["path"].apply(path_to_bounds)
bounds_gdf = gpd.GeoDataFrame(df, geometry="bounds", crs=4326)
bounds_gdf.head()

joined = gpd.sjoin(bounds_gdf, regions, how="inner")
paths = joined["path"].unique()
paths

array(['/cccr-lab/000_data-repository/008_fabdem/tiles/N08E149_FABDEM_V1-2.tif',
       '/cccr-lab/000_data-repository/008_fabdem/tiles/N08E150_FABDEM_V1-2.tif'],
      dtype=object)

In [15]:
import rioxarray as rxr
import geopandas as gpd
import os
from tqdm import tqdm
from glob import glob
from spatialoperations.types.xrtypes import RasterLike
import xarray as xr

def safe_get_region(path):
    try:
        ds = rxr.open_rasterio(path).isel(band=0)
        init_crs = ds.rio.crs
        ds = xr.where(ds > 0, ds, np.nan)
        ds.rio.write_crs(init_crs, inplace=True)
        return RasterLike(ds, name=path.split('/')[-1].split('.')[0])
    except Exception as e:
        return None
        
FABDEM = {
    f"idx_{idx}": safe_get_region(path)
    for idx, path in tqdm(enumerate(paths))
}

FABDEM = {
    k: v for k, v in FABDEM.items() if v is not None
}

FABDEM
# FABDEM.keys()


2it [00:00,  2.79it/s]


{'idx_0': RasterLike(_data=<xarray.DataArray (y: 3600, x: 3600)> Size: 52MB
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
 Coordinates:
     band         int64 8B 1
   * x            (x) float64 29kB 149.0 149.0 149.0 149.0 ... 150.0 150.0 150.0
   * y            (y) float64 29kB 9.0 9.0 8.999 8.999 ... 8.001 8.001 8.001 8.0
     spatial_ref  int64 8B 0, name='N08E149_FABDEM_V1-2'),
 'idx_1': RasterLike(_data=<xarray.DataArray (y: 3600, x: 3600)> Size: 52MB
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]], d

In [16]:
from spatialoperations import rasterops
from spatialoperations.rasterops import intake
from spatialoperations import compute

jlc = compute.JoblibCompute(n_jobs=20)

so.ro.zarr_manager.root_group.create_group("topobathy/topography")
so.ro.intake.import_multi_raster_to_zarr_with_prep(
    FABDEM, 
    "topobathy/topography", 
    output_path="FABDEM",
    compute=jlc,
)

100%|█████████████████████████████████████████| 4/4 [00:00<00:00, 42799.02it/s]
